In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# Import CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "snhu305"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME 
#Add in Grazioso Salvare’s logo
image_filename = 'grazioso_salvare_logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

# Add your unique identifier
creator_identifier = "Created by Vivian Nguyen"

app.layout = html.Div([
    ## Logo and hyperlink
    # Logo and hyperlink
    html.Div([
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'height': '10%', 'width': '10%'}
            ),
            href='https://www.snhu.edu'
        ),
        
        # Creator identifier
        html.H1(
            creator_identifier,
            style={'font-size': '14px', 'margin-left': '0px', 'font-style': 'italic'}  # Adjust font size and margin as needed
        )
    ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'space-between'}),  # Align logo and identifier horizontally
    
    html.Hr(),
    
    # Title of the dashboard
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    
    # Content of the dashboard...      
    #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    # Interactive filtering options
    html.Div([
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',  # Default selected value
            clearable=False  # Disable option to clear selection
        )
    ]),
    html.Hr(),

    #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
    #If you completed the Module Six Assignment, you can copy in the code you created here 

    # dash_table.DataTable component
    dash_table.DataTable(
        id='datatable-id',  # Unique identifier for the DataTable component
        columns=[  # Column definition
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            # Loop through DataFrame columns to generate DataTable columns
        ],
        data=df.to_dict('records'),  # Data to be displayed in the table, converted to a dictionary format
        editable=True,
        filter_action="native",  # Enables native filtering functionality
        sort_action="native",  # Enables native sorting functionality
        sort_mode="multi",  # Allows multi-column sorting
        selected_columns=[],  # Initially selected columns (empty list)
        selected_rows=[0],  # Initially selected rows (empty list)
        page_action="native",  # Enables native pagination
        page_current=0,  # Current page index (zero-based)
        page_size=10,  # Number of rows per page
    ),

    html.Br(),
    html.Hr(),

    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',

                ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                )
            ]
        )
])

#############################################
# Interaction Between Components / Controller
#############################################
# Define the filter query construction function
def construct_filter_query(filter_type):
    # Define the Rescue Type, Preferred Breeds, Preferred Sex, and Training Age range mapping
    preferred_info_mapping = {
        'Water Rescue': {
            'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
            'sex': 'Intact Female',
            'age_range': (26, 156)  # Training Age range in weeks
        },
        'Mountain or Wilderness Rescue': {
            'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
            'sex': 'Intact Male',
            'age_range': (26, 156)  # Training Age range in weeks
        },
        'Disaster Rescue or Individual Tracking': {
            'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
            'sex': 'Intact Male',
            'age_range': (20, 300)  # Training Age range in weeks
        }
    }
    
    # Get preferred breeds, sex, and age range based on the selected rescue type
    preferred_breeds = preferred_info_mapping[filter_type]['breeds']
    preferred_sex = preferred_info_mapping[filter_type]['sex']
    min_age, max_age = preferred_info_mapping[filter_type]['age_range']
    
    # Construct the filter query
    filter_query = {
        '$and': [
            {'breed': {'$in': preferred_breeds}},
            {'sex_upon_outcome': preferred_sex},
            {'age_upon_outcome_in_weeks': {'$gte': min_age, '$lte': max_age}}  # Age range query
        ]
    }
    
    return filter_query

   
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
# def update_dashboard(filter_type):
# ## FIX ME Add code to filter interactive data table with MongoDB queries
# #
# #        
# #        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
# #        data=df.to_dict('records')
# #       
# #       
# #        return (data,columns)
    
def update_dashboard(filter_type):
    if filter_type == 'Reset':
#         print(df.columns)
        return df.to_dict('records')
    else:
        # Construct the filter query based on the selected rescue type
        filter_query = construct_filter_query(filter_type)
        
        # Query the Animal Shelter database with the filter query
        filtered_data = pd.DataFrame.from_records(db.read(filter_query))  # Query the MongoDB collection
        filtered_data.drop(columns=['_id'],inplace=True)
#         print(filtered_data.columns)  
        return filtered_data.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    # Check if viewData is not None
    if viewData is None:
        return []

    # Convert the virtual data to a DataFrame
    df = pd.DataFrame(viewData)

    # Count the occurrences of each breed
    breed_counts = df['breed'].value_counts()

    # Create a pie chart using Plotly Express
    fig = px.pie(names=breed_counts.index, values=breed_counts.values, title='Preferred Animals')

    # Return the pie chart as a dcc.Graph component
    return dcc.Graph(figure=fig)
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', 'active_cell')])
def update_map(viewData, active_cell):

    if active_cell is None:
        # Return an empty map
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[dl.TileLayer(id="base-layer-id")])
    
    # Extract the row index from the active_cell
    row = active_cell['row']

    # Create the map with marker for the selected row
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
        center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
    ])
]   



app.run_server(debug=True)


Connection Sucessful
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:22846/
